In [1]:
import rutils

In [2]:
rutils.memento()  # use memento to authenticate if it exists


In [3]:
host = "cmedge003.usmsc18.pie.apple.com"
conn = rutils.RemoteFSHook(host)

In [ ]:
host = "cmedge003.usmsc18.pie.apple.com"
conn = rutils.RemoteFSHook(host)

In [4]:
conn.home

PosixPath('/srv/home/tchen47')

In [5]:
conn.tmux_run_command(command="ls -l ~/", session_name="test")
conn.tmux_kill_session("test")

<Result cmd='tmux kill-session -t test' exited=0>

In [6]:
html = "<h1>Test report</h1>"
rutils.send_email_message(
    from_email="Test Report <no-reply@apple.com>",
    to_emails=["tchen47@apple.com"],
    subject="Test report",
    content=html,
)

In [ ]:
scp -r usmsc18-3:/srv/home/tchen47/HLB_fixWindow_trial_raw_data_test.csv ~/Desktop/local_filename/

In [11]:
conn.download("/srv/home/tchen47/HLB_fixWindow_trial_raw_data_test.csv", "~/Desktop")

PosixPath('/Users/tuochen/Desktop/HLB_fixWindow_trial_raw_data_test.csv')

In [12]:
import datetime
import rutils

end_date = datetime.datetime.utcnow() + datetime.timedelta(seconds=30)

def my_func():
    """Returns true after it reaches the end_date in 30 seconds."""
    now = datetime.datetime.utcnow()
    if now >= end_date:
        return rutils.PokeReturnValue(True, "Finished")
    else:
        return rutils.PokeReturnValue(False, "Waiting")

rutils.python_sensor(func=my_func)
# 15:30:00 : INFO : Poking callable: my_func
# 15:30:00 : INFO : Sleeping for 60 seconds
# 15:31:00 : INFO : Poking callable: my_func
# 15:31:00 : INFO : Success criteria met. Exiting.
print(out)  # Finished

NameError: name 'out' is not defined

In [13]:
import logging
from pathlib import Path
from rocketry.conds import after_success, every
import rutils

In [14]:
log_file = Path("./logs/rutils.log")
log_file.parent.mkdir(exist_ok=True, parents=True)
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s | %(name)s | %(levelname)s | %(message)s",
    handlers=[logging.FileHandler(log_file), logging.StreamHandler()],
)

In [15]:
scheduler = rutils.Scheduler()

In [16]:
@scheduler.task(start_cond=every("30 seconds"))
def start():
    logging.info(f"Starting stage 1")

In [17]:
@scheduler.task(start_cond=after_success(start) | every("10 seconds"))
def polling():
    logging.info("Polling")

In [18]:
from pathlib import Path
import rutils

host = "cmedge003.usmsc18.pie.apple.com"
run_id = "my-script"
local_file = Path("./script.scala")

In [19]:
rutils.memento()
conn = rutils.RemoteFSHook(host)
spark = rutils.SparkOperator(conn=conn)

In [20]:
info = spark.run(
    local_file=local_file,
    run_id=run_id, 
    # properties are saved to a file on the remote machine and can be 
    # read by the script
    properties=dict(
        start_date="2022-11-21",
        end_date="2022-11-28"
    ), 
    spark_binary="spark.shell2"
)

FileNotFoundError: File script.scala does not exist

In [21]:
import logging
from pathlib import Path
import datetime
import rutils

rutils.memento()

log_file = Path("./logs/rutils.log")
log_file.parent.mkdir(exist_ok=True, parents=True)
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s | %(name)s | %(levelname)s | %(message)s",
    handlers=[logging.FileHandler(log_file), logging.StreamHandler()],
)

In [22]:
host = "cmedge003.usmsc18.pie.apple.com"
conn = rutils.RemoteFSHook(host)
hdfs = rutils.RemoteHDFSHook(conn=conn)
spark = rutils.SparkOperator(conn=conn)

In [23]:
hdfs_output_dir = Path("/user/arahman22/weather_data_monitoring/raw_data.parquet/")
hdfs_output_file = hdfs_output_dir / "{date}.parquet"

In [24]:
days = 10
start_date = datetime.date(2023, 1, 10)
backfill_dates = [start_date + datetime.timedelta(days=i) for i in range(days)]

In [25]:
def date_file_exists(date: datetime.datetime) -> bool:
    file = str(hdfs_output_file).format(date=date)
    return rutils.PokeReturnValue(
        is_done=hdfs.exists(file), 
        value=file
    )